In [1]:
import OpenDartReader
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta
from html_table_parser import parser_functions as parser
import xlwings as xw
from tqdm import tqdm
import FinanceDataReader as fdr
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from selenium import webdriver
import os

api_key = '08d5ae18b24d9a11b7fd67fb0d79c607f1c88464'
dart = OpenDartReader(api_key)
today = datetime.today().strftime("%Y%m%d") #오늘

corp_code = '368770'
corp_name = '파이버프로'
start = '2020-01-02'
dart_list = dart.list(corp_code,start=start)
lists = dart_list[dart_list['report_nm'].str.contains('공급계약체결')]
print(f'{len(lists)}개 조회')
def supply(rcp_no,corp):
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    driver = webdriver.Chrome(options=options)
    driver.get(f'https://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcp_no}')
    driver.switch_to.frame('ifrm')
    html = driver.page_source #지금 현 상태의 page source불러오기
    soup = BeautifulSoup(html,'html.parser')
    tables = soup.find_all('table')#표를 불러옴
    p=parser.make2d(tables[0])#표를 만듬
    tt=pd.DataFrame(p)
    tt = tt[[1,2]]
    tt = tt.T
    
    if '내용' in tt.iloc[0,0]: # 공급계약은 두가지 타입으로 존재
        tt = tt[[19,0,2,5,6,7,12,13,14,21,20]]
        tt.columns = ['계약일자','계약내용','계약금액(원)','최근매출액(원)','매출액대비(%)','계약상대','판매공급지역','시작일','종료일','유보사유','유보기한']
        tt = tt.drop(tt.index[0]).reset_index()
        tt = tt.drop(columns='index')
    else:
        tt = tt[[12,1,2,3,4,6,8,9,10,13,14]]
        tt.columns = ['계약일자','계약내용','계약금액(원)','최근매출액(원)','매출액대비(%)','계약상대','판매공급지역','시작일','종료일','유보사유','유보기한']
        tt = tt.drop(tt.index[0]).reset_index()
        tt = tt.drop(columns='index')
    tt.insert(0,'기업',corp)
    tt.insert(0,'공시구분','최초공시')
    tt.insert(0,'조회날짜',today)
    time.sleep(1)
    return tt

def supply_adj(rcp_no,corp):
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    driver = webdriver.Chrome(options=options)
    driver.get(f'https://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcp_no}')
    driver.switch_to.frame('ifrm')
    html = driver.page_source #지금 현 상태의 page source불러오기
    soup = BeautifulSoup(html,'html.parser')
    tables = soup.find_all('table')#표를 불러옴
    p=parser.make2d(tables[-1])#표를 만듬
    tt=pd.DataFrame(p)
    tt = tt[[1,2]]
    tt = tt.T
    
    if '내용' in tt.iloc[0,0]: # 공급계약은 두가지 타입으로 존재
        tt = tt[[19,0,2,5,6,7,12,13,14,21,20]]
        tt.columns = ['계약일자','계약내용','계약금액(원)','최근매출액(원)','매출액대비(%)','계약상대','판매공급지역','시작일','종료일','유보사유','유보기한']
        tt = tt.drop(tt.index[0]).reset_index()
        tt = tt.drop(columns='index')
    else:
        tt = tt[[12,1,2,3,4,6,8,9,10,13,14]]
        tt.columns = ['계약일자','계약내용','계약금액(원)','최근매출액(원)','매출액대비(%)','계약상대','판매공급지역','시작일','종료일','유보사유','유보기한']
        tt = tt.drop(tt.index[0]).reset_index()
        tt = tt.drop(columns='index')
    tt.insert(0,'기업',corp)
    tt.insert(0,'공시구분','기재정정')
    tt.insert(0,'조회날짜',today)
    time.sleep(1)
    return tt

# df = pd.DataFrame()

# for i in range(len(lists)):
#     try:
#         rcp_no = lists['rcept_no'].values[i]
#         corp = lists['flr_nm'].values[i]
#         supply_df = supply(rcp_no,corp)
        
#         print(f'{i}_{corp} 완료')  
#         df = pd.concat([df,supply_df])
#     except Exception as e:
#         print(f'{corp}',e)


df = pd.DataFrame()
for i,re_nm in lists.iterrows():
    try:
        if '기재정정' in re_nm['report_nm']:
            rcp_no = re_nm['rcept_no']
            corp = re_nm['corp_name']
            equipment_df = supply_adj(rcp_no,corp) # 기재정정
            print(f'기재정정 {corp}_{i}건')
            df = pd.concat([df,equipment_df])
        else:
            rcp_no = re_nm['rcept_no']
            corp = re_nm['corp_name']   
            equipment_df = supply(rcp_no,corp) # 최초공시
            print(f'최초공시 {corp}_{i}건')
            df = pd.concat([df,equipment_df])
    except Exception as e:
        print(re_nm['corp_name'],e)

folder = f'공급계약_개별기업'
file_name = f'공급계약_{corp_name}_20200102 ~ 20241026.xlsx'
file_path = os.path.join(folder,file_name)


if not os.path.exists(folder):
    os.mkdir(folder)
    
if not os.path.exists(file_path):
    with open(file_path, 'w') as f:
        df.to_excel(file_path)



13개 조회
기재정정 파이버프로_1건
최초공시 파이버프로_3건
최초공시 파이버프로_16건
최초공시 파이버프로_17건
최초공시 파이버프로_24건


KeyboardInterrupt: 

In [3]:
df.to_clipboard()

Empty DataFrame
Columns: []
Index: []